### Version 1 - High Level Overview

In [12]:
from image_prep import get_img_df, train_val_test_split
from image_gen import get_gen_from_df

In [ ]:
fc_path = "data/Food_Classification/"

# get df
df_fc = get_img_df(fc_path)

num_classes = len(df_fc['label'].unique())
num_classes

In [14]:
# split dfs
train, validate, test = train_val_test_split(df_fc, test_size=0.2, val_size=0.2)

In [ ]:
train_gen = get_gen_from_df(train)
validate_gen = get_gen_from_df(validate, train=False)

The following model is probably not the final model we want to use but just needed something to test and end to end flow.  

This particular architecture was borrowed from the Jupyter notebook found: https://www.kaggle.com/code/varsha300/transferlearning


In [16]:
from tensorflow.keras.applications import VGG16
from tensorflow.keras.applications.vgg16 import preprocess_input
from tensorflow.keras.layers import Flatten, Dense, Dropout, BatchNormalization, Input
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam

# Constants
IMG_SIZE = (224, 224)  # VGG16 default image size

# model

base_model = VGG16(weights='imagenet', include_top=False, input_tensor=Input(shape=(IMG_SIZE[0], IMG_SIZE[1], 3)))
# Freeze base model layers and unfreeze the last 4 layers
for layer in base_model.layers[:-4]:
    layer.trainable = False

# Model architecture
x = base_model.output
x = Flatten()(x)
x = Dense(4096, activation='relu')(x)
x = Dropout(0.5)(x)
x = BatchNormalization()(x)
x = Dense(4096, activation='relu')(x)
x = Dropout(0.5)(x)
predictions = Dense(num_classes, activation='softmax')(x)
model = Model(inputs=base_model.input, outputs=predictions)

# Compile model
model.compile(optimizer=Adam(learning_rate=1e-5), loss='categorical_crossentropy', metrics=['accuracy'])


In [ ]:
# Train model
history = model.fit(train_gen, validation_data=validate_gen, epochs=1, steps_per_epoch=train_gen.num_batches)


In [ ]:
# Evaluate the model on the validation set
val_loss, val_accuracy = model.evaluate(validate_gen)
print(f'Validation accuracy: {val_accuracy * 100:.2f}%')

### Version 2 - Even Simpler

In [ ]:
import cv_models 

%load_ext autoreload
%autoreload 2

In [ ]:
from image_prep import get_split_data
df_train, df_validate, df_test = get_split_data()
train_gen, validate_gen = cv_models.get_training_data(df_train, df_validate, 32)

type(train_gen)

In [ ]:
best_acc, best_model_name, best_dropout, all_results = cv_models.find_best_model(10)